In [33]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn import cross_validation as cv

#  Reading files and merging dataframes

In [ ]:
# read csv of yelp_business.csv
yelp_business_df = pd.read_csv('C:\Users\Kai Liang\Desktop\capstone\yelp_business.csv')

# read csv of yelp_reviews.csv
yelp_review_df = pd.read_csv('C:\Users\Kai Liang\Desktop\capstone\yelp_review.csv')
yelp_review_df.head()

# merging the dataframes of reviews and business
result = pd.merge(yelp_review_df, yelp_business_df, how='outer', on=['business_id'])

# observing only restaurants from the categories column in yelp dataset
list_of_restaurants =  ['Restaurants' in x for x in result['categories']]
restaurants_df = result[list_of_restaurants]
restaurants_df.head()

# taking only restuarants in Montreal to analyse
restaurants_MT = restaurants_df[restaurants_df.city == 'Montréal']
restaurants_MT.reset_index(inplace=True)
restaurants_MT = restaurants_MT.drop('index',axis =1)


In [79]:
restaurants_MT.head()

,review_id,user_id,business_id,stars_x,date,text,useful,funny,cool,name,...,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,categories
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0,"""Wilensky's""",...,"""34 Avenue Fairmount Ouest""",Montréal,QC,H2T 2M1,45.523333,-73.594859,4.0,84,1,Diners;Food;Restaurants;Delis
1,vm1b1keOzwHjtGZEPPuYXA,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5,2016-02-22,*The shop was featured on Mind of a Chef with ...,4,3,4,"""Wilensky's""",...,"""34 Avenue Fairmount Ouest""",Montréal,QC,H2T 2M1,45.523333,-73.594859,4.0,84,1,Diners;Food;Restaurants;Delis
2,SXwA9KZ-Nc_hMARk_3cJ7g,5Ymfsf9fAYz-Ds_p0xawVQ,AEx2SYEUJmTxVVB18LlCwA,5,2013-03-29,"As its name suggests, Wilensky's is a lunch co...",9,0,3,"""Wilensky's""",...,"""34 Avenue Fairmount Ouest""",Montréal,QC,H2T 2M1,45.523333,-73.594859,4.0,84,1,Diners;Food;Restaurants;Delis
3,jUzausdZ_ujqe_n8BlBj-g,DVOOF0Z627DyrZ4XKQbTgA,AEx2SYEUJmTxVVB18LlCwA,5,2017-08-08,"Stopped by for the Wilensky's Special, loved i...",0,0,0,"""Wilensky's""",...,"""34 Avenue Fairmount Ouest""",Montréal,QC,H2T 2M1,45.523333,-73.594859,4.0,84,1,Diners;Food;Restaurants;Delis
4,oCRDwF3tszAkeszSfxwthg,5JoKz3mU42Cp906KRXDwJw,AEx2SYEUJmTxVVB18LlCwA,4,2009-01-17,"I have to admit, I am a fan. Just entering the...",4,0,1,"""Wilensky's""",...,"""34 Avenue Fairmount Ouest""",Montréal,QC,H2T 2M1,45.523333,-73.594859,4.0,84,1,Diners;Food;Restaurants;Delis


In [80]:
# look at the shape
print restaurants_MT.shape

# take a look at the dataframe to ensure that it is in order.
restaurants_MT.info()

(93359, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93359 entries, 0 to 93358
Data columns (total 21 columns):
review_id       93359 non-null object
user_id         93359 non-null object
business_id     93359 non-null object
stars_x         93359 non-null int64
date            93359 non-null object
text            93359 non-null object
useful          93359 non-null int64
funny           93359 non-null int64
cool            93359 non-null int64
name            93359 non-null object
neighborhood    93172 non-null object
address         93359 non-null object
city            93359 non-null object
state           93359 non-null object
postal_code     93264 non-null object
latitude        93359 non-null float64
longitude       93359 non-null float64
stars_y         93359 non-null float64
review_count    93359 non-null int64
is_open         93359 non-null int64
categories      93359 non-null object
dtypes: float64(3), int64(6), object(12)
memory usage: 15.0+ MB


In [81]:
#extract only the user_id, business_id and stars_x which is the user rating.
restaurants_MT_CF = restaurants_MT[['user_id', 'business_id', 'stars_x']]

In [82]:
restaurants_MT_CF.head()

,user_id,business_id,stars_x
0,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5
1,xYciRtVZ1PW4IxSX4oJ1aw,AEx2SYEUJmTxVVB18LlCwA,5
2,5Ymfsf9fAYz-Ds_p0xawVQ,AEx2SYEUJmTxVVB18LlCwA,5
3,DVOOF0Z627DyrZ4XKQbTgA,AEx2SYEUJmTxVVB18LlCwA,5
4,5JoKz3mU42Cp906KRXDwJw,AEx2SYEUJmTxVVB18LlCwA,4


In [83]:
# get a dataframe of users with more than 10 reviews 

vc = restaurants_MT_CF.user_id.value_counts()

list_of_users = vc[vc >10]

df1 = pd.DataFrame(list_of_users )
df1 = df1.reset_index()
df1.columns = ['user_id','count']
df1

,user_id,count
0,Wc5L6iuvSNF5WGBlqIO8nw,384
1,qKpkRCPk4ycbllTfFcRbNw,267
2,3aYeG-x5A44GIgmBHrwyAA,220
3,6MM9Yqn7UBM8tmpSHQHAAg,205
4,La1ggB37gvq6UWpEWaMtMA,184
5,UDFIZXP0kOb2xKoYToTS6w,170
6,U1vl4SQzO3wTAWlYVnSjnw,169
7,XOgqs74hkbXT54mSp7JMRA,159
8,ILe3tzgnbGSaZfpArtJdjA,158
9,ZD76B53WiEdv3g2lNgTbNg,148


In [84]:
df1.shape

(1135, 2)

In [85]:
#merge the dataframes
result = pd.merge(restaurants_MT_CF, df1, how='left', on='user_id')
# drop those users that have less than 10 reviews
restaurants_MT_CF = result.dropna()

In [86]:
restaurants_MT_CF.head()

,user_id,business_id,stars_x,count
2,5Ymfsf9fAYz-Ds_p0xawVQ,AEx2SYEUJmTxVVB18LlCwA,5,26.0
3,DVOOF0Z627DyrZ4XKQbTgA,AEx2SYEUJmTxVVB18LlCwA,5,11.0
8,NGLufCod1pP760aBucp3tw,AEx2SYEUJmTxVVB18LlCwA,3,79.0
9,mYQmaTeWp87sGnE-rHvIOw,AEx2SYEUJmTxVVB18LlCwA,5,34.0
11,06hNNsVZO78El38qNJaETA,AEx2SYEUJmTxVVB18LlCwA,5,20.0


#  Spilt to training and testing data matrix

In [87]:
# spilt the data to train and test data
train_data, test_data = cv.train_test_split(restaurants_MT_CF, test_size=0.2)


In [88]:
print train_data.shape
print test_data.shape

(25720, 4)
(6431, 4)


In [89]:
#Create two user-item matrices, one for training and another for testing

# first create the train data
train_data_matrix = train_data.pivot_table(values = 'stars_x', index = 'user_id', columns = 'business_id')  

# # mean centre
# train_data_matrix = train_data_matrix.apply(lambda x: x-x.mean(),axis =1)

# Replace NaNs with 0s
train_data_matrix.fillna(0, inplace = True)

 
# secondly create the test data
test_data_matrix = test_data.pivot_table(values = 'stars_x', index = 'user_id', columns = 'business_id')  

# # mean centre
# test_data_matrix = test_data_matrix.apply(lambda x: x-x.mean(),axis =1)

# Replace NaNs with 0s
test_data_matrix.fillna(0, inplace = True)


In [90]:
# make to array
train_data_matrix = train_data_matrix.values
test_data_matrix = test_data_matrix.values

# Cosine similarity calculations for CF

In [91]:
# calculating the cosine_similarity of User User and Item Item

user_similarity = cosine_similarity(train_data_matrix)
item_similarity = cosine_similarity(train_data_matrix.T)

In [92]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [93]:
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')

# Evaluating the CF model 

In [94]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore zero terms values
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))


In [95]:
# RMSE on the test data
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.9158837691
Item-based CF RMSE: 3.9009207281


In [96]:
# RMSE on the train data
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, train_data_matrix)))

User-based CF RMSE: 3.47152516754
Item-based CF RMSE: 3.40905936155


In [97]:
#predicting for CF models 


# Evaluating the SVD model 

In [98]:
from surprise import Reader, Dataset

reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(restaurants_MT_CF[['user_id', 'business_id', 'stars_x']], reader)

# Split data into 5 folds

data.split(n_folds=5)


from surprise import SVD, evaluate
from surprise import NMF, KNNBasic

# svd Mean RMSE: 0.9766
algo = SVD()
evaluate(algo, data, measures=['RMSE'])

# # nmf Mean RMSE: 1.0924
# algo = NMF()
# evaluate(algo, data, measures=['RMSE'])

# #KNNBasic Mean RMSE: 1.0715
# algo = KNNBasic() 
# evaluate(algo, data, measures=['RMSE'])

C:\anaconda\envs\python2\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\anaconda\envs\python2\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.9653
------------
Fold 2
RMSE: 0.9761
------------
Fold 3
RMSE: 0.9811
------------
Fold 4
RMSE: 0.9942
------------
Fold 5
RMSE: 0.9815
------------
------------
Mean RMSE: 0.9796
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9653381256769874,
                             0.9760596953087958,
                             0.9810672305633755,
                             0.9941985258670618,
                             0.9815122921418339]})

# Predicting for SVD models 

In [101]:
# Retrieve the trainset.
trainset = data.build_full_trainset()
algo.train(trainset)

C:\anaconda\envs\python2\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


In [102]:
restaurants_MT_CF = restaurants_MT_CF.reset_index().drop(['count','index'],axis=1)

In [103]:
restaurants_MT_CF

,user_id,business_id,stars_x
0,5Ymfsf9fAYz-Ds_p0xawVQ,AEx2SYEUJmTxVVB18LlCwA,5
1,DVOOF0Z627DyrZ4XKQbTgA,AEx2SYEUJmTxVVB18LlCwA,5
2,NGLufCod1pP760aBucp3tw,AEx2SYEUJmTxVVB18LlCwA,3
3,mYQmaTeWp87sGnE-rHvIOw,AEx2SYEUJmTxVVB18LlCwA,5
4,06hNNsVZO78El38qNJaETA,AEx2SYEUJmTxVVB18LlCwA,5
5,0TJ01iwzVLXtr_EYcfqoWQ,AEx2SYEUJmTxVVB18LlCwA,5
6,aU6Mq536aI1r_BunJE-V2w,AEx2SYEUJmTxVVB18LlCwA,4
7,FWo1hSUugKWFQwr5IpCANw,AEx2SYEUJmTxVVB18LlCwA,5
8,XduciX-f9ONDZ-rUhs3PfQ,AEx2SYEUJmTxVVB18LlCwA,3
9,RxPndFw7sdSX4hWMbIJBGQ,AEx2SYEUJmTxVVB18LlCwA,4


In [169]:
#selecting a random user to test
x = np.random.randint(len(restaurants_MT_CF))
users= restaurants_MT_CF['user_id']
random_user = users.iloc[x]
#selecting a random restaurant from the random user to test
y = np.random.randint(len(restaurants_MT_CF[restaurants_MT_CF['user_id'] == random_user]))
titles= restaurants_MT_CF[restaurants_MT_CF['user_id'] == random_user]['business_id']
stars =  restaurants_MT_CF[restaurants_MT_CF['user_id'] == random_user]['stars_x']
random_restaurant = titles.iloc[y]
stars = stars.iloc[y]

# Predict a certain item

userid = random_user
itemid = random_restaurant
actual_rating = stars
print(algo.predict(userid, itemid, actual_rating))



user: FQo8HxOxlhMgDQod9NwfHQ item: f-GwGqmGJ17F6w5CXcZkSg r_ui = 4.00   est = 3.79   {u'was_impossible': False}


In [170]:
rand_df = restaurants_MT_CF[restaurants_MT_CF['user_id'] == random_user]
rand_df[rand_df['business_id'] == random_restaurant]

,user_id,business_id,stars_x
18850,FQo8HxOxlhMgDQod9NwfHQ,f-GwGqmGJ17F6w5CXcZkSg,4
